In [1]:
import sqlite3
from faker import Faker
import random
import pandas as pd
import time

In [2]:
fake = Faker()

In [14]:
conn = sqlite3.connect('e-commerce.db')
cursor = conn.cursor()

In [4]:
def insert_CONSUMER_data(n):
    for i in range(n):
        cursor.execute("""
            INSERT INTO CONSUMER (Email, Name, Birth_date, Phone_number, Password, Country, Street, Postcode, Building, City)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            fake.unique.email(),
            fake.first_name(),
            fake.date_of_birth(minimum_age=18, maximum_age=80),
            fake.random_int(min=1000000000, max=9999999999),
            fake.password(length=10),
            fake.country(),
            fake.street_name(),
            fake.postcode(),
            fake.building_number(),
            fake.city()
        ))

        if (i + 1) % 50 == 0:
            conn.commit()
            print(f"{i + 1} CONSUMER records committed.")
            time.sleep(0.5)  # 等待

    conn.commit()
    print(f"Total of {n} records inserted successfully into CONSUMER table.")

# Insert CONSUMER data
insert_CONSUMER_data(200)

50 CONSUMER records committed.
100 CONSUMER records committed.
150 CONSUMER records committed.
200 CONSUMER records committed.
Total of 200 records inserted successfully into CONSUMER table.


In [5]:
# Step 2: Insert data into PRODUCT from CSV
df_product = pd.read_csv('Product_data.csv')
df_product.to_sql('PRODUCT', conn, if_exists='append', index=False)
conn.commit()
print("Product data inserted.")

Product data inserted.


In [6]:
# Step 3: Insert data into PAYMENT
cursor.execute("SELECT Email FROM CONSUMER")
consumer_emails = [row[0] for row in cursor.fetchall()]

def insert_payment_data(emails):
    for i, email in enumerate(emails):
        payment_id = i + 1
        payment_type = random.choice(["CREDIT_DEBIT", "VOUCHERS_GIFT"])

        cursor.execute("""
            INSERT INTO PAYMENT (Payment_ID, Payment_type, Email)
            VALUES (?, ?, ?)
        """, (payment_id, payment_type, email))

        if (i + 1) % 50 == 0:
            conn.commit()
            time.sleep(1.0)
            print(f"{i + 1} PAYMENT records committed.")

    conn.commit()
    print(f"{len(emails)} PAYMENT records inserted.")

# Insert PAYMENT data using unique emails from the consumer list
insert_payment_data(consumer_emails)

50 PAYMENT records committed.
100 PAYMENT records committed.
150 PAYMENT records committed.
200 PAYMENT records committed.
200 PAYMENT records inserted.


In [7]:
from datetime import datetime
import random

# Retrieve all Payment_IDs and Consumer Names where Payment_type is 'CREDIT_DEBIT'
cursor.execute("""
    SELECT PAYMENT.Payment_ID, CONSUMER.Name
    FROM PAYMENT
    JOIN CONSUMER ON PAYMENT.Email = CONSUMER.Email
    WHERE PAYMENT.Payment_type = 'CREDIT_DEBIT'
""")
credit_debit_records = cursor.fetchall()

# Insert data into CREDIT_DEBIT for each PAYMENT ID with Payment_type as 'CREDIT_DEBIT'
for i, (payment_id, consumer_name) in enumerate(credit_debit_records):
    card_number = fake.unique.random_int(min=1000000000000000, max=9999999999999999)  # 16-digit card number
    verification_code = random.randint(100, 999)  # 3-digit verification code
    is_default = random.randint(0, 1)  # Default card flag, 0 or 1
    
    # Generate an expiry date in the future with a random year (1-5 years from now) and a random month
    current_year = datetime.now().year
    expiry_year = current_year + random.randint(1, 5)
    expiry_month = random.randint(1, 12)
    expiry_date = f"{expiry_year}-{expiry_month:02d}-1"  

    cursor.execute("""
        INSERT INTO CREDIT_DEBIT (Card_number, Verification_code, Name, Is_default, Expiry_date, Payment_ID)
        VALUES (?, ?, ?, ?, ?, ?)
    """, (card_number, verification_code, consumer_name, is_default, expiry_date, payment_id))
    
    # Commit every 50 records and add a short delay
    if (i + 1) % 50 == 0:
        conn.commit()
        time.sleep(1.0)  # Pause to reduce load on the database
        print(f"{i + 1} CREDIT_DEBIT records committed.")

# Final commit for any remaining records
conn.commit()
print(f"{len(credit_debit_records)} CREDIT_DEBIT records inserted.")

50 CREDIT_DEBIT records committed.
97 CREDIT_DEBIT records inserted.


In [8]:
from datetime import datetime, timedelta

# Retrieve all Payment_IDs where Payment_type is 'VOUCHERS_GIFT'
cursor.execute("""
    SELECT PAYMENT.Payment_ID
    FROM PAYMENT
    JOIN CONSUMER ON PAYMENT.Email = CONSUMER.Email
    WHERE PAYMENT.Payment_type = 'VOUCHERS_GIFT'
""")
vouchers_gift_records = cursor.fetchall()

# Insert data into VOUCHERS_GIFT for each PAYMENT ID with Payment_type as 'VOUCHERS_GIFT'
for i, record in enumerate(vouchers_gift_records):
    payment_id = record[0]
    serial_number = fake.unique.random_int(min=10000000000000, max=99999999999999)  # 14-digit serial number
    total_amount = random.choice([x for x in range(1000, 3001, 100)])  # Choose from 1000, 1100, ..., 3000
    current_balance = round(random.uniform(0, total_amount), 2)  # Balance with two decimal places, max as total_amount
    expiry_date = datetime.now() + timedelta(days=random.randint(365, 730))  # Expiry date 1 to 2 years from now

    cursor.execute("""
        INSERT INTO VOUCHERS_GIFT (Serial_number, Total_amount, Current_balance, Expiry_date, Payment_ID)
        VALUES (?, ?, ?, ?, ?)
    """, (serial_number, total_amount, current_balance, expiry_date, payment_id))
    
    # Commit every 50 records and add a short delay
    if (i + 1) % 50 == 0:
        conn.commit()
        time.sleep(0.1)
        print(f"{i + 1} VOUCHERS_GIFT records committed.")

# Final commit for any remaining records
conn.commit()
print(f"{len(vouchers_gift_records)} VOUCHERS_GIFT records inserted.")

50 VOUCHERS_GIFT records committed.
100 VOUCHERS_GIFT records committed.
103 VOUCHERS_GIFT records inserted.


In [9]:
# Retrieve all Product_numbers from the PRODUCT table
cursor.execute("SELECT Product_number FROM PRODUCT")
product_numbers = [row[0] for row in cursor.fetchall()]

# Retrieve all Emails from the CONSUMER table
cursor.execute("SELECT Email FROM CONSUMER")
consumer_emails = [row[0] for row in cursor.fetchall()]

# Function to insert data into the BASKET table, ensuring each consumer has a unique basket
def insert_basket_data(consumer_emails):
    for i, email in enumerate(consumer_emails):
        # Randomly select 1 to 10 Product_numbers for each basket
        selected_products = random.sample(product_numbers, k=random.randint(1, 10))
        product_list = ','.join(map(str, selected_products))  # Convert list of product numbers to a comma-separated string
        
        # Basket_ID can be generated based on the index, assuming it's unique
        basket_id = i + 1

        # Insert data into the BASKET table for each consumer email
        cursor.execute("""
            INSERT INTO BASKET (Basket_ID, Product_list, Email)
            VALUES (?, ?, ?)
        """, (basket_id, product_list, email))
        
        # Commit every 50 records and add a short delay
        if (i + 1) % 50 == 0:
            conn.commit()
            time.sleep(2.0)
            print(f"{i + 1} Basket records committed.")

    # Commit any remaining records
    conn.commit()
    print(f"{len(consumer_emails)} records successfully inserted into the BASKET table with product numbers.")

# Call the function to insert baskets for each consumer
insert_basket_data(consumer_emails)

50 Basket records committed.
100 Basket records committed.
150 Basket records committed.
200 Basket records committed.
200 records successfully inserted into the BASKET table with product numbers.


In [15]:

# Retrieve all necessary data for generating orders
cursor.execute("""
    SELECT CONSUMER.Email, BASKET.Product_list, PAYMENT.Payment_ID 
    FROM CONSUMER 
    JOIN BASKET ON CONSUMER.Email = BASKET.Email 
    JOIN PAYMENT ON CONSUMER.Email = PAYMENT.Email
""")
consumer_data = cursor.fetchall()

# Function to insert orders
def insert_orders(n):
    for i in range(n):
        # Select a random consumer
        email, product_list, payment_id = random.choice(consumer_data)
        
        # Generate a unique order number
        order_number = i + 1
        
        # Generate an order date within this year
        order_date = fake.date_this_year()
        
        # Split the product list by commas to get available product numbers for this consumer
        product_numbers = product_list.split(',')
        
        # Select a subset or all of the products to be included in this order
        selected_products = random.sample(product_numbers, k=random.randint(1, len(product_numbers)))
        
        # Join the selected products back into a single string for the order's product list
        order_product_list = ','.join(selected_products)
        
        # Calculate the subtotal for the order by summing the subtotals of selected products
        cursor.execute(f"""
            SELECT SUM(Subtotal) 
            FROM Product_Subtotal_View
            WHERE Product_number IN ({', '.join(['?'] * len(selected_products))})
        """, selected_products)
        
        subtotal = cursor.fetchone()[0]
        
        # Check if subtotal is None and handle accordingly
        if subtotal is None:
            print(f"Warning: Subtotal for order {order_number} with products {selected_products} is None.")
            continue  # Skip this order if subtotal couldn't be calculated
        
        # Apply a random discount or promotion
        deduction_promotion = round(random.uniform(0, 50), 2)  # Random discount between £0 and £50
        grand_total = subtotal - deduction_promotion
        
        # Insert the order into the ORDERS table
        cursor.execute("""
            INSERT INTO ORDERS (Order_number, Order_date, Product, Subtotal, Deduction_Promotion, Email, Payment_ID)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        """, (order_number, order_date, order_product_list, subtotal, deduction_promotion, email, payment_id))
        
        # Commit every 50 records and add a short delay
        if (i + 1) % 50 == 0:
            conn.commit()
            time.sleep(1.0)
            print(f"{i + 1} orders committed.")
    
    # Final commit for any remaining records
    conn.commit()
    print(f"{n} orders successfully inserted into the ORDERS table.")

# Insert 220 orders
insert_orders(220)


50 orders committed.
100 orders committed.
150 orders committed.
200 orders committed.
220 orders successfully inserted into the ORDERS table.


In [19]:
from datetime import timedelta, datetime

# Retrieve data from the ConsumerOrderView
cursor.execute("SELECT Country, Street, Postcode, Building, City, Order_number, Order_date FROM ConsumerOrderView")
order_data = cursor.fetchall()

# Set the number of deliveries and returns
delivery_count = 170  # Updated to 170 deliveries
return_count = len(order_data) - delivery_count

# Split the data into delivery and return records
delivery_orders = random.sample(order_data, delivery_count)
return_orders = [order for order in order_data if order not in delivery_orders]

# Start Track_number at 100000000 and increment
track_number_start = 100000000

# Function to insert data into the DELIVERY table
def insert_delivery_data(delivery_orders):
    for i, record in enumerate(delivery_orders):
        country, street, postcode, building, city, order_number, order_date_str = record
        
        # Convert order_date to datetime if needed
        order_date = datetime.strptime(order_date_str, '%Y-%m-%d') if isinstance(order_date_str, str) else order_date_str
        
        # Calculate the delivery date
        delivery_date = order_date + timedelta(days=random.randint(1, 15))
        
        # Build Customer_address
        customer_address = f"{street}, {building}, {city}, {postcode}, {country}"
        
        # Set one random status to 1 (delivered, postponed, cancelled, pending)
        statuses = [0, 0, 0, 0]
        statuses[random.randint(0, 3)] = 1
        delivered, postponed, cancelled, pending = statuses
        
        # Insert into DELIVERY table with incremented Track_number
        cursor.execute("""
            INSERT INTO DELIVERY (Track_number, Delivery_date, Customer_address, Delivered, Postponed, Cancelled, Pending, Order_number)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """, (track_number_start + i, delivery_date, customer_address, delivered, postponed, cancelled, pending, order_number))
        
        # Commit every 50 records
        if (i + 1) % 50 == 0:
            conn.commit()
            print(f"{i + 1} DELIVERY records committed.")

    conn.commit()
    time.sleep(1)
    print(f"{len(delivery_orders)} records successfully inserted into the DELIVERY table.")

# Insert delivery data
insert_delivery_data(delivery_orders)


50 DELIVERY records committed.
100 DELIVERY records committed.
150 DELIVERY records committed.
170 records successfully inserted into the DELIVERY table.


In [20]:
from datetime import timedelta, datetime

# Retrieve necessary fields from ConsumerOrderView for return orders
cursor.execute("""
    SELECT Order_number, Order_date, Grand_total 
    FROM ConsumerOrderView
""")
order_data = cursor.fetchall()

# Set the number of returns you want
return_count = 50  # Adjust as needed for the number of returns

# Select a subset of orders for returns
return_orders = random.sample(order_data, return_count)

# Set the starting point for Ticket_number
ticket_number_start = 1000000  # Starting point for 7-digit Ticket_number

# Function to insert data into the RETURN table
def insert_return_data(return_orders):
    for i, record in enumerate(return_orders):
        order_number, order_date_str, grand_total = record
        
        # Convert order_date to datetime if needed
        start_date = datetime.strptime(order_date_str, '%Y-%m-%d') if isinstance(order_date_str, str) else order_date_str
        due_date = start_date + timedelta(days=random.randint(7, 30))
        
        # Generate a unique sequential 7-digit Ticket_number
        ticket_number = ticket_number_start + i
        
        # Refund_total is set to Grand_total from ConsumerOrderView
        refund_total = grand_total
        
        # Randomly assign one status (completed, cancelled, denied, pending) as 1
        statuses = [0, 0, 0, 0]
        statuses[random.randint(0, 3)] = 1
        completed, cancelled, denied, pending = statuses
        
        # Insert into RETURN table
        cursor.execute("""
            INSERT INTO RETURN (Ticket_number, Start_date, Due_date, Refund_total, Completed, Cancelled, Denied, Pending, Order_number)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (ticket_number, start_date, due_date, refund_total, completed, cancelled, denied, pending, order_number))
        
        # Commit every 50 records to avoid locking issues
        if (i + 1) % 50 == 0:
            conn.commit()
            print(f"{i + 1} RETURN records committed.")

    conn.commit()
    print(f"{len(return_orders)} records successfully inserted into the RETURN table.")

# Insert return data
insert_return_data(return_orders)

50 RETURN records committed.
50 records successfully inserted into the RETURN table.


In [21]:
import random
from datetime import timedelta, datetime

# Retrieve data from ConsumerOrderView, excluding orders already in DELIVERY
cursor.execute("""
    SELECT o.Order_number, o.Grand_total, o.Order_date
    FROM ConsumerOrderView o
    LEFT JOIN DELIVERY d ON o.Order_number = d.Order_number
    WHERE d.Order_number IS NULL
""")
return_orders = cursor.fetchall()

# Function to insert data into the RETURN table
def insert_return_data(return_orders):
    for i, record in enumerate(return_orders):
        order_number, grand_total, order_date_str = record
        
        # Convert order_date to datetime if needed
        start_date = datetime.strptime(order_date_str, '%Y-%m-%d') if isinstance(order_date_str, str) else order_date_str
        due_date = start_date + timedelta(days=random.randint(7, 30))
        
        # Generate a unique 7-digit Ticket_number
        ticket_number = 1000000 + i  # Sequential 7-digit number starting from 1000000
        
        # Set refund_total to the grand_total from ConsumerOrderView
        refund_total = grand_total
        
        # Set one random status to 1
        statuses = [0, 0, 0, 0]
        statuses[random.randint(0, 3)] = 1
        completed, cancelled, denied, pending = statuses
        
        # Insert into RETURN table
        cursor.execute("""
            INSERT INTO RETURN (Ticket_number, Start_date, Due_date, Refund_total, Completed, Cancelled, Denied, Pending, Order_number)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (ticket_number, start_date, due_date, refund_total, completed, cancelled, denied, pending, order_number))
        
        # Commit every 50 records
        if (i + 1) % 50 == 0:
            conn.commit()
            print(f"{i + 1} RETURN records committed.")

    conn.commit()
    print(f"{len(return_orders)} records successfully inserted into the RETURN table.")

# Insert data into the RETURN table
insert_return_data(return_orders)

50 RETURN records committed.
50 records successfully inserted into the RETURN table.


In [22]:
# Retrieve all basket data (Basket_ID and Product_list) from the BASKET table
cursor.execute("SELECT Basket_ID, Product_list FROM BASKET")
basket_data = cursor.fetchall()

# Function to insert data into the CONTAINS table
def insert_contains_data(basket_data):
    for basket_id, product_list in basket_data:
        # Split the product list to get individual product numbers and count occurrences
        product_numbers = product_list.split(',')
        
        # Count the occurrences of each product number
        product_counts = {}
        for product_number in product_numbers:
            product_number = product_number.strip()  # Remove any extra spaces
            if product_number in product_counts:
                product_counts[product_number] += 1
            else:
                product_counts[product_number] = 1

        # Insert each product and its respective quantity into the CONTAINS table
        for product_number, respective_quantity in product_counts.items():
            cursor.execute("""
                INSERT INTO contains (Basket_ID, Product_number, Respective_quantity)
                VALUES (?, ?, ?)
            """, (basket_id, int(product_number), respective_quantity))
        
        # Commit every 50 baskets and add a short delay
        if basket_id % 50 == 0:
            conn.commit()
            time.sleep(0.5)
            print(f"{basket_id} baskets processed.")

    # Final commit for any remaining records
    conn.commit()
    print("All records successfully inserted into the contains table.")

# Call the function to populate the CONTAINS table
insert_contains_data(basket_data)

50 baskets processed.
100 baskets processed.
150 baskets processed.
200 baskets processed.
All records successfully inserted into the contains table.


In [23]:
# Retrieve all order data (Order_number and Product) from the ORDERS table
cursor.execute("SELECT Order_number, Product FROM ORDERS")
order_data = cursor.fetchall()

# Function to insert data into the INCLUDES table
def insert_includes_data(order_data):
    for order_number, product_list in order_data:
        # Split the comma-separated product numbers list
        product_numbers = product_list.split(',')
        
        # Count occurrences of each product to determine respective quantities
        product_counts = {}
        for product_number in product_numbers:
            product_counts[product_number] = product_counts.get(product_number, 0) + 1

        # Insert each product with its respective quantity into the INCLUDES table
        for product_number, quantity in product_counts.items():
            cursor.execute("""
                INSERT INTO includes (Order_number, Product_number, Quantity)
                VALUES (?, ?, ?)
            """, (order_number, int(product_number), quantity))

        # Commit every 50 records and add a short delay
        if order_number % 50 == 0:
            conn.commit()
            time.sleep(0.5)
            print(f"{order_number} orders processed.")

    # Final commit for any remaining records
    conn.commit()
    print("All records successfully inserted into the includes table.")

# Call the function to populate the INCLUDES table
insert_includes_data(order_data)

50 orders processed.
100 orders processed.
150 orders processed.
200 orders processed.
All records successfully inserted into the includes table.


In [24]:
import pandas as pd
import sqlite3

# Load the CSV data into a DataFrame
review_data = pd.read_csv('Review_table.csv')

# Connect to the SQLite database
conn = sqlite3.connect('E-commerce.db')

# Insert the data into the REVIEW table
review_data.to_sql('REVIEW', conn, if_exists='append', index=False)

# Commit and close the connection
conn.commit()

print("Data from review_table.csv successfully inserted into the REVIEW table.")

Data from review_table.csv successfully inserted into the REVIEW table.


In [25]:
conn.close()